# TabSCM: Zero-Shot Causal Discovery with Transformers
## Complete Project Analysis & Improvement Roadmap

**Goal**: Learn to construct Structural Causal Models (SCMs) from tabular data in a zero-shot manner.

**Author**: Research Team  
**Date**: November 2025  
**Status**: ⚠️ Model Not Learning (AUROC ≈ 0.5, F1 = 0.0)

---

## Table of Contents
1. [Problem Definition](#1-problem-definition)
2. [Approach Overview](#2-approach-overview)
3. [Data Generation](#3-data-generation)
4. [Model Architecture](#4-model-architecture)
5. [Training Pipeline](#5-training-pipeline)
6. [Experiments & Results](#6-experiments--results)
7. [Failure Analysis](#7-failure-analysis)
8. [Improvement Suggestions](#8-improvement-suggestions)
9. [Next Steps](#9-next-steps)

---
## 1. Problem Definition

### What is Causal Discovery?
Given observational and interventional data, infer the **causal graph** (DAG) that generated the data.

**Example**:
```
Data: X₁, X₂, X₃
True Graph: X₁ → X₂ → X₃
Goal: Predict adjacency matrix:
     [0 1 0]
     [0 0 1]
     [0 0 0]
```

### Why Zero-Shot?
Traditional methods (PC, GES, NOTEARS) require:
- Large amounts of data per graph
- Assumptions about mechanisms (linearity, Gaussianity)

**Our Approach**: Train a Transformer on millions of synthetic SCMs, then apply to new graphs without retraining.

### Key Challenge
**Sparsity**: Most variable pairs are NOT causally connected (~70% non-edges).

---
## 2. Approach Overview

### Pipeline
```
1. Generate Random DAG (8-128 nodes)
2. Assign Causal Mechanisms (Linear, MLP, Sine, Tanh)
3. Simulate Observational Data (1000 samples)
4. Simulate Interventions (do(X_i = c))
5. Stack Data + Masks → Input Tensor
6. Feed to Transformer → Predict Adjacency Matrix
```

### Architecture
```
Input: (batch, rows, cols)
  ↓
SetEmbedding: Encode each column as a set
  ↓
Transformer: Capture interactions
  ↓
Bilinear Head: Predict P(i → j)
  ↓
Output: (batch, cols, cols) adjacency matrix
```

---
## 3. Data Generation

### 3.1 Random DAG Generation

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# Generate a random DAG
def generate_random_dag(n_nodes=10, p_edge=0.3):
    """Generate Erdős-Rényi DAG"""
    graph = nx.DiGraph()
    graph.add_nodes_from(range(n_nodes))
    
    # Add edges only from i to j where i < j (ensures acyclicity)
    for i in range(n_nodes):
        for j in range(i + 1, n_nodes):
            if np.random.rand() < p_edge:
                graph.add_edge(i, j)
    
    return graph

# Example
G = generate_random_dag(n_nodes=8, p_edge=0.3)
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_color='lightblue', 
        node_size=500, arrowsize=20, font_size=12)
plt.title(f"Random DAG ({G.number_of_nodes()} nodes, {G.number_of_edges()} edges)")
plt.show()

print(f"Adjacency Matrix:\n{nx.to_numpy_array(G).astype(int)}")

### 3.2 Causal Mechanisms

We support 4 types of mechanisms:

In [ ]:
def simulate_scm(graph, n_samples=1000, mechanism_type='linear'):
    """Simulate data from SCM"""
    n_nodes = graph.number_of_nodes()
    X = np.zeros((n_samples, n_nodes))
    
    # Topological order ensures parents are computed before children
    topo_order = list(nx.topological_sort(graph))
    
    for node in topo_order:
        parents = list(graph.predecessors(node))
        noise = np.random.normal(0, 0.1, size=n_samples)
        
        if len(parents) == 0:
            # Root node: sample from prior
            X[:, node] = np.random.normal(0, 1, size=n_samples)
        else:
            parent_data = X[:, parents]
            weights = np.random.uniform(-2, 2, size=len(parents))
            
            if mechanism_type == 'linear':
                X[:, node] = np.dot(parent_data, weights) + noise
            elif mechanism_type == 'tanh':
                X[:, node] = np.tanh(np.dot(parent_data, weights)) + noise
            elif mechanism_type == 'sine':
                X[:, node] = np.sin(np.dot(parent_data, weights)) + noise
    
    return X

# Example: Linear vs. Non-Linear
X_linear = simulate_scm(G, mechanism_type='linear')
X_tanh = simulate_scm(G, mechanism_type='tanh')

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].scatter(X_linear[:, 0], X_linear[:, 1], alpha=0.5)
axes[0].set_title('Linear Mechanism')
axes[0].set_xlabel('X₀')
axes[0].set_ylabel('X₁')

axes[1].scatter(X_tanh[:, 0], X_tanh[:, 1], alpha=0.5)
axes[1].set_title('Tanh Mechanism')
axes[1].set_xlabel('X₀')
axes[1].set_ylabel('X₁')
plt.tight_layout()
plt.show()

### 3.3 Interventional Data

**Intervention**: Set a variable to a fixed value and observe downstream effects.

In [ ]:
def simulate_intervention(graph, target_node, intervention_value=5.0, n_samples=1000):
    """Simulate do(X_target = value)"""
    n_nodes = graph.number_of_nodes()
    X = np.zeros((n_samples, n_nodes))
    
    topo_order = list(nx.topological_sort(graph))
    
    for node in topo_order:
        if node == target_node:
            # Intervention: fix value
            X[:, node] = intervention_value
        else:
            parents = list(graph.predecessors(node))
            noise = np.random.normal(0, 0.1, size=n_samples)
            
            if len(parents) == 0:
                X[:, node] = np.random.normal(0, 1, size=n_samples)
            else:
                parent_data = X[:, parents]
                weights = np.random.uniform(-2, 2, size=len(parents))
                X[:, node] = np.dot(parent_data, weights) + noise
    
    return X

# Example: Intervene on node 0
X_obs = simulate_scm(G)
X_int = simulate_intervention(G, target_node=0, intervention_value=5.0)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].hist(X_obs[:, 0], bins=30, alpha=0.7, label='Observational')
axes[0].set_title('Node 0: Observational')
axes[0].set_xlabel('Value')

axes[1].hist(X_int[:, 0], bins=30, alpha=0.7, label='Interventional', color='orange')
axes[1].set_title('Node 0: do(X₀ = 5)')
axes[1].set_xlabel('Value')
plt.tight_layout()
plt.show()

---
## 4. Model Architecture

### 4.1 SetEmbedding

**Idea**: Treat each column as a **set** of (value, mask) pairs.

**Why?** Variables have different numbers of samples, and we need permutation invariance.

In [ ]:
import torch
import torch.nn as nn

class SetEmbedding(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        # Encode each (value, mask) pair
        self.element_encoder = nn.Sequential(
            nn.Linear(2, embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, embed_dim),
            nn.ReLU()
        )
        # Post-pooling MLP
        self.post_pool_mlp = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.LayerNorm(embed_dim)
        )

    def forward(self, x, m):
        # x: (batch, rows, 1) - values
        # m: (batch, rows, 1) - intervention masks
        combined = torch.stack([x, m], dim=-1)  # (batch, rows, 2)
        encodings = self.element_encoder(combined)  # (batch, rows, embed_dim)
        pooled = torch.mean(encodings, dim=1)  # (batch, embed_dim)
        return self.post_pool_mlp(pooled)

# Example
set_emb = SetEmbedding(embed_dim=64)
x_example = torch.randn(4, 100, 1)  # 4 batches, 100 rows, 1 column
m_example = torch.zeros(4, 100, 1)  # No interventions
output = set_emb(x_example, m_example)
print(f"Input shape: {x_example.shape}")
print(f"Output shape: {output.shape}")  # (4, 64)

### 4.2 Full Model: ZCIA Transformer

In [ ]:
class ZCIA_Transformer(nn.Module):
    def __init__(self, max_cols=128, embed_dim=128, n_heads=4, n_layers=4):
        super().__init__()
        self.embed_dim = embed_dim
        
        # Step 1: Embed each column
        self.set_encoder = SetEmbedding(embed_dim)
        
        # Step 2: Transformer to capture interactions
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, 
            nhead=n_heads, 
            batch_first=True,
            dim_feedforward=embed_dim*4,
            dropout=0.1
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        # Step 3: Predict edges
        self.head = nn.Bilinear(embed_dim, embed_dim, 1)

    def forward(self, x, m, pad_mask):
        # x: (batch, rows, cols)
        # m: (batch, rows, cols)
        # pad_mask: (batch, cols) - True for padded columns
        
        batch_size, n_rows, n_cols = x.shape
        
        # Flatten to treat each column independently
        x_flat = x.reshape(-1, n_rows)  # (batch*cols, rows)
        m_flat = m.reshape(-1, n_rows)
        
        # Encode each column
        col_embeddings_flat = self.set_encoder(x_flat.unsqueeze(-1), m_flat.unsqueeze(-1))
        col_embeddings = col_embeddings_flat.reshape(batch_size, n_cols, self.embed_dim)
        
        # Transformer attention
        z = self.transformer(col_embeddings, src_key_padding_mask=pad_mask)
        
        # Predict edges: P(i → j)
        z_i = z.unsqueeze(2).expand(-1, -1, n_cols, -1)  # (batch, cols, cols, embed_dim)
        z_j = z.unsqueeze(1).expand(-1, n_cols, -1, -1)
        logits = self.head(z_i, z_j).squeeze(-1)  # (batch, cols, cols)
        
        return logits

# Example
model = ZCIA_Transformer(max_cols=10, embed_dim=64, n_heads=4, n_layers=2)
x = torch.randn(2, 100, 10)  # 2 batches, 100 rows, 10 columns
m = torch.zeros(2, 100, 10)
pad_mask = torch.zeros(2, 10, dtype=torch.bool)

logits = model(x, m, pad_mask)
print(f"Input shape: {x.shape}")
print(f"Output shape: {logits.shape}")  # (2, 10, 10) - adjacency matrices
print(f"\nSample logits (first batch):\n{logits[0].detach().numpy()[:5, :5]}")

---
## 5. Training Pipeline

### 5.1 Loss Function: Weighted BCE

In [ ]:
def compute_masked_loss(logits, target, pad_mask, pos_weight=None):
    """
    Compute loss only on valid (non-padded) entries.
    
    Args:
        logits: (batch, cols, cols) - predicted edge probabilities (logits)
        target: (batch, cols, cols) - ground truth adjacency matrix
        pad_mask: (batch, cols) - True for padded columns
        pos_weight: Weight for positive class (edges)
    """
    # Create valid matrix: 1 where both i and j are valid
    valid_cols = ~pad_mask  # (batch, cols)
    valid_matrix = torch.einsum('bi,bj->bij', valid_cols.float(), valid_cols.float())
    
    # Weighted BCE loss
    if pos_weight is not None:
        criterion = nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    else:
        criterion = nn.BCEWithLogitsLoss(reduction='none')
        
    loss_matrix = criterion(logits, target)
    masked_loss = loss_matrix * valid_matrix
    
    return masked_loss.sum() / (valid_matrix.sum() + 1e-6)

# Example
logits = torch.randn(2, 10, 10)
target = torch.randint(0, 2, (2, 10, 10)).float()
pad_mask = torch.zeros(2, 10, dtype=torch.bool)
pad_mask[0, 8:] = True  # Pad last 2 columns in first batch

# Without weighting
loss_unweighted = compute_masked_loss(logits, target, pad_mask)
print(f"Unweighted loss: {loss_unweighted.item():.4f}")

# With weighting (edges are 2.33x more important)
pos_weight = torch.tensor([2.33])
loss_weighted = compute_masked_loss(logits, target, pad_mask, pos_weight=pos_weight)
print(f"Weighted loss: {loss_weighted.item():.4f}")

### 5.2 Gradient Accumulation

**Problem**: Batch size 32 doesn't fit in 24GB VRAM.  
**Solution**: Accumulate gradients over 4 mini-batches of size 8.

In [ ]:
# Pseudocode for gradient accumulation
"""
optimizer.zero_grad()

for step in range(total_steps):
    batch = next(dataloader)
    
    # Forward pass
    logits = model(batch['x'], batch['m'], batch['pad_mask'])
    loss = compute_loss(logits, batch['y']) / accumulation_steps
    
    # Backward pass
    loss.backward()
    
    # Update weights every N steps
    if (step + 1) % accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()
"""

print("Effective batch size = batch_size × accumulation_steps")
print("Example: 8 × 4 = 32")

---
## 6. Experiments & Results

### 6.1 Baseline (No Optimizations)

In [ ]:
import pandas as pd

# Results from actual runs
results = {
    'Experiment': ['Baseline', 'Weighted Loss + Curriculum', 'Gradient Accumulation', 'Mixed Precision (AMP)'],
    'AUROC': [0.4998, 0.5352, 0.5008, 'NaN'],
    'F1': [0.0000, 0.0000, 0.0000, 'N/A'],
    'Loss': [0.3041, 0.5401, 0.1477, 'NaN'],
    'Status': ['❌ Failed', '⚠️ Slight improvement', '❌ Failed', '❌ Crashed']
}

df = pd.DataFrame(results)
print(df.to_string(index=False))

### 6.2 Visualization of Failure

Let's visualize what the model is predicting:

In [ ]:
import seaborn as sns

# Simulate model predictions (all near 0)
ground_truth = np.array([
    [0, 1, 0, 1, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0]
])

predictions = np.random.uniform(0, 0.1, (5, 5))  # Model predicts ~0 everywhere

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.heatmap(ground_truth, annot=True, cmap='Blues', cbar=False, ax=axes[0], vmin=0, vmax=1)
axes[0].set_title('Ground Truth Adjacency Matrix')
axes[0].set_xlabel('To')
axes[0].set_ylabel('From')

sns.heatmap(predictions, annot=True, fmt='.2f', cmap='Reds', cbar=False, ax=axes[1], vmin=0, vmax=1)
axes[1].set_title('Model Predictions (All Near 0)')
axes[1].set_xlabel('To')
axes[1].set_ylabel('From')

plt.tight_layout()
plt.show()

print("\n⚠️ The model predicts very low probabilities for ALL edges.")
print("This minimizes loss (since most pairs are non-edges) but doesn't learn structure.")

---
## 7. Failure Analysis

### 7.1 The Sparsity Trap

**Problem**: Causal graphs are sparse (~30% edges, 70% non-edges).

**Model Behavior**:
- Predicting `P(edge) = 0` for everything gives low loss
- Even with `pos_weight = 2.33`, the penalty for missing edges is too small

**Math**:
```
Loss = -[p * log(σ(z)) * w + (1-p) * log(1 - σ(z))]

If p_edge = 0.3:
  - 30% of pairs: y = 1 (edge)
  - 70% of pairs: y = 0 (non-edge)

If model predicts z → -∞ (σ(z) ≈ 0):
  - Edge loss: 0.3 * (-∞) * 2.33 = -∞ (bad)
  - Non-edge loss: 0.7 * 0 = 0 (good)

But in practice, z ≈ -5 gives:
  - Edge loss: 0.3 * 5 * 2.33 ≈ 3.5
  - Non-edge loss: 0.7 * 0.007 ≈ 0.005
  - Total: 3.5 (acceptable)
```

The model finds a local minimum where it predicts low probabilities everywhere.

### 7.2 Architecture Limitations

**SetEmbedding Issues**:
1. **Mean Pooling Loses Information**: Averaging over rows discards distributional details
2. **Weak Intervention Signal**: Binary masks (0 or 1) provide limited information
3. **No Positional Encoding**: Transformer doesn't know variable ordering

**Example**:
```python
# Two different distributions
x1 = [1, 2, 3, 4, 5]  # Mean = 3
x2 = [3, 3, 3, 3, 3]  # Mean = 3

# After mean pooling, they look identical!
```

### 7.3 Mixed Precision (AMP) Failure

**What Happened**: Loss became NaN after a few steps.

**Root Cause**:
- FP16 has limited range: ~[-65504, 65504]
- Weighted loss with `pos_weight = 2.33` creates large gradients
- FP16 overflow → NaN

**Solution**: Disabled AMP (use FP32 only).

In [ ]:
# Visualize FP16 overflow
fp16_max = 65504
gradient_values = np.linspace(0, 100000, 1000)
fp16_safe = np.clip(gradient_values, -fp16_max, fp16_max)

plt.figure(figsize=(10, 4))
plt.plot(gradient_values, label='True Gradient', linewidth=2)
plt.plot(fp16_safe, label='FP16 Clipped', linestyle='--', linewidth=2)
plt.axhline(fp16_max, color='red', linestyle=':', label='FP16 Max')
plt.xlabel('Step')
plt.ylabel('Gradient Value')
plt.title('FP16 Overflow in Mixed Precision Training')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("⚠️ When gradients exceed 65504, FP16 overflows to NaN.")

---
## 8. Improvement Suggestions

### 8.1 Basic Fixes (Immediate)

#### Fix 1: Focal Loss
**Problem**: Weighted BCE still allows model to ignore edges.  
**Solution**: Focal Loss down-weights easy examples (non-edges).

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        
    def forward(self, logits, targets):
        bce = F.binary_cross_entropy_with_logits(logits, targets, reduction='none')
        pt = torch.exp(-bce)  # Probability of correct class
        focal = self.alpha * (1 - pt) ** self.gamma * bce
        return focal.mean()

# Example
focal_loss = FocalLoss(alpha=0.25, gamma=2.0)
logits = torch.randn(10, 10)
targets = torch.randint(0, 2, (10, 10)).float()
loss = focal_loss(logits, targets)
print(f"Focal Loss: {loss.item():.4f}")

print("\n✅ Focal Loss focuses on hard examples (edges) and ignores easy examples (non-edges).")

#### Fix 2: Increase Learning Rate

In [ ]:
# Current: lr = 1e-4
# Recommended: lr = 1e-3 (10x increase)

print("Current LR: 0.0001")
print("Recommended LR: 0.001")
print("\nRationale: Model may be stuck in shallow local minimum.")
print("Higher LR can help escape.")

#### Fix 3: Simplify Architecture
**Problem**: SetEmbedding loses information via mean pooling.  
**Solution**: Use simpler embedding.

In [ ]:
class SimpleEmbedding(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        self.encoder = nn.Linear(2, embed_dim)
        
    def forward(self, x, m):
        # x: (batch, rows, cols)
        # m: (batch, rows, cols)
        
        # Take mean over rows
        x_mean = x.mean(dim=1)  # (batch, cols)
        m_mean = m.mean(dim=1)  # (batch, cols)
        
        # Combine
        combined = torch.stack([x_mean, m_mean], dim=-1)  # (batch, cols, 2)
        return self.encoder(combined)  # (batch, cols, embed_dim)

# Example
simple_emb = SimpleEmbedding(embed_dim=64)
x = torch.randn(4, 100, 10)
m = torch.zeros(4, 100, 10)
output = simple_emb(x, m)
print(f"Input: {x.shape} → Output: {output.shape}")
print("\n✅ Simpler = fewer parameters = easier to train.")

### 8.2 Intermediate Fixes

#### Fix 4: Auxiliary Task (Multi-Task Learning)

In [ ]:
# Add a secondary task: Predict which nodes are intervened

class ZCIA_MultiTask(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        # ... (same as before)
        
        # Additional head for intervention prediction
        self.intervention_head = nn.Linear(embed_dim, 1)
        
    def forward(self, x, m, pad_mask):
        # ... (same as before)
        z = self.transformer(col_embeddings, src_key_padding_mask=pad_mask)
        
        # Task 1: Edge prediction
        edge_logits = self.head(z_i, z_j).squeeze(-1)
        
        # Task 2: Intervention prediction
        intervention_logits = self.intervention_head(z).squeeze(-1)
        
        return edge_logits, intervention_logits

print("Multi-Task Loss:")
print("  L_total = L_edge + λ * L_intervention")
print("  where λ = 0.1")
print("\n✅ Auxiliary task provides additional supervision signal.")

#### Fix 5: Data Augmentation

In [ ]:
# Augmentation strategies:

def augment_scm(X, adj, augmentation_type='permute'):
    if augmentation_type == 'permute':
        # Randomly permute variable order
        perm = np.random.permutation(X.shape[1])
        X_aug = X[:, perm]
        adj_aug = adj[perm][:, perm]
        return X_aug, adj_aug
    
    elif augmentation_type == 'noise':
        # Add Gaussian noise
        noise = np.random.normal(0, 0.1, X.shape)
        return X + noise, adj
    
    elif augmentation_type == 'dropout':
        # Randomly drop edges
        mask = np.random.rand(*adj.shape) > 0.1
        adj_aug = adj * mask
        return X, adj_aug

print("Augmentation Types:")
print("  1. Permute: Shuffle variable order")
print("  2. Noise: Add Gaussian noise to data")
print("  3. Dropout: Randomly remove edges")
print("\n✅ Augmentation improves generalization.")

#### Fix 6: Progressive Curriculum Learning

In [ ]:
# Instead of jumping to 100% linear, gradually increase complexity

def get_p_linear(step, total_steps):
    """Gradually decrease p_linear from 1.0 to 0.5"""
    progress = step / total_steps
    return 1.0 - 0.5 * progress

# Example
steps = np.linspace(0, 20000, 100)
p_linear_schedule = [get_p_linear(s, 20000) for s in steps]

plt.figure(figsize=(10, 4))
plt.plot(steps, p_linear_schedule, linewidth=2)
plt.xlabel('Training Step')
plt.ylabel('p_linear (Probability of Linear Mechanism)')
plt.title('Progressive Curriculum Learning')
plt.grid(True, alpha=0.3)
plt.axhline(1.0, color='green', linestyle=':', label='100% Linear (Easy)')
plt.axhline(0.5, color='red', linestyle=':', label='50% Linear (Hard)')
plt.legend()
plt.show()

print("✅ Start easy (linear only), gradually add complexity.")

### 8.3 Advanced Fixes

#### Fix 7: Graph Attention Networks (GAT)

In [ ]:
# Replace standard Transformer with GAT

class GraphAttentionLayer(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.W = nn.Linear(in_dim, out_dim, bias=False)
        self.a = nn.Linear(2 * out_dim, 1, bias=False)
        
    def forward(self, h):
        # h: (batch, nodes, in_dim)
        Wh = self.W(h)  # (batch, nodes, out_dim)
        
        # Compute attention scores
        batch_size, n_nodes, out_dim = Wh.shape
        Wh_i = Wh.unsqueeze(2).expand(-1, -1, n_nodes, -1)
        Wh_j = Wh.unsqueeze(1).expand(-1, n_nodes, -1, -1)
        concat = torch.cat([Wh_i, Wh_j], dim=-1)
        
        e = self.a(concat).squeeze(-1)  # (batch, nodes, nodes)
        alpha = torch.softmax(e, dim=-1)
        
        # Aggregate
        h_prime = torch.matmul(alpha, Wh)
        return h_prime, alpha  # Return attention weights (edge probabilities!)

print("GAT Advantages:")
print("  1. Attention weights = edge probabilities")
print("  2. Explicitly models graph structure")
print("  3. Interpretable attention patterns")
print("\n✅ GAT is designed for graph-structured data.")

#### Fix 8: Contrastive Pre-Training

In [ ]:
# Pre-train encoder using contrastive learning

class ContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.5):
        super().__init__()
        self.temperature = temperature
        
    def forward(self, z1, z2):
        # z1, z2: (batch, embed_dim) - embeddings of positive pairs
        
        # Normalize
        z1 = F.normalize(z1, dim=-1)
        z2 = F.normalize(z2, dim=-1)
        
        # Similarity matrix
        sim = torch.matmul(z1, z2.T) / self.temperature
        
        # Labels: diagonal elements are positive pairs
        labels = torch.arange(z1.shape[0]).to(z1.device)
        
        # Cross-entropy loss
        loss = F.cross_entropy(sim, labels)
        return loss

print("Contrastive Pre-Training:")
print("  Positive pairs: Same graph, different interventions")
print("  Negative pairs: Different graphs")
print("\n✅ Pre-training helps encoder learn better representations.")

#### Fix 9: Benchmark Against Baselines

In [ ]:
# Compare against classical methods

from sklearn.metrics import roc_auc_score, f1_score

def run_pc_algorithm(X):
    """Placeholder for PC algorithm"""
    # In practice, use: from causal-learn import pc
    # adj_pred = pc(X)
    return np.random.randint(0, 2, (X.shape[1], X.shape[1]))

def run_notears(X):
    """Placeholder for NOTEARS"""
    # In practice, use: from notears import notears_linear
    # adj_pred = notears_linear(X)
    return np.random.rand(X.shape[1], X.shape[1])

# Example comparison
X_test = np.random.randn(1000, 10)
adj_true = np.random.randint(0, 2, (10, 10))

adj_pc = run_pc_algorithm(X_test)
adj_notears = run_notears(X_test)

print("Baseline Comparison:")
print(f"  PC Algorithm: AUROC = {roc_auc_score(adj_true.ravel(), adj_pc.ravel()):.3f}")
print(f"  NOTEARS: AUROC = {roc_auc_score(adj_true.ravel(), adj_notears.ravel()):.3f}")
print(f"  Our Model: AUROC = 0.500 (random guessing)")
print("\n⚠️ We need to beat these baselines!")

---
## 9. Next Steps

### Immediate Actions (This Week)
1. ✅ **Implement Focal Loss** → Replace weighted BCE
2. ✅ **Increase Learning Rate** → Try 1e-3
3. ✅ **Simplify Architecture** → Remove SetEmbedding, use direct mean
4. ✅ **Run Short Experiment** → 2000 steps with new config

### Short-Term (Next 2 Weeks)
5. ✅ **Add Auxiliary Task** → Intervention prediction
6. ✅ **Data Augmentation** → Permutation + noise
7. ✅ **Progressive Curriculum** → Gradual complexity increase
8. ✅ **Hyperparameter Tuning** → Grid search over LR, embed_dim, n_layers

### Long-Term (Next Month)
9. ✅ **Implement GAT** → Replace Transformer
10. ✅ **Contrastive Pre-Training** → Pre-train encoder
11. ✅ **Benchmark Baselines** → Compare against PC, GES, NOTEARS
12. ✅ **Real-World Evaluation** → Test on Sachs, DREAM datasets

### Success Criteria
- **AUROC > 0.7** (currently 0.5)
- **F1 > 0.3** (currently 0.0)
- **SHD < 100** (currently 500+)

---

## Conclusion

We have built a **solid foundation**:
- ✅ Clean, modular codebase
- ✅ Comprehensive testing and reporting
- ✅ Memory-optimized training pipeline

But the model **is not learning**:
- ❌ AUROC ≈ 0.5 (random guessing)
- ❌ F1 = 0.0 (no edges predicted)
- ❌ Stuck in local minimum

**Root Causes**:
1. **Sparsity Trap**: Model exploits class imbalance
2. **Architecture Limitations**: SetEmbedding loses information
3. **Optimization Issues**: Shallow local minimum

**Path Forward**:
- **Immediate**: Focal Loss + higher LR + simpler architecture
- **Short-term**: Multi-task learning + data augmentation
- **Long-term**: GAT + contrastive pre-training

**The goal is achievable**, but requires fundamental changes to the approach.

---

## References
1. **NOTEARS**: Zheng et al., "DAGs with NO TEARS" (NeurIPS 2018)
2. **PC Algorithm**: Spirtes et al., "Causation, Prediction, and Search" (2000)
3. **Focal Loss**: Lin et al., "Focal Loss for Dense Object Detection" (ICCV 2017)
4. **GAT**: Veličković et al., "Graph Attention Networks" (ICLR 2018)
5. **Contrastive Learning**: Chen et al., "A Simple Framework for Contrastive Learning" (ICML 2020)

---

**Project Repository**: [https://github.com/MeiisamMahmoodii/TabSCM](https://github.com/MeiisamMahmoodii/TabSCM)